In [1]:
#Dependencies
import requests
import json
from api_keys import api_key_FBI
import pandas as pd

In [5]:
# the abbreviation of every state.
state_codes = pd.DataFrame({"Abbreviation":['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT',"DC",
                'DE', 'FL',"FS", 'GA',"GM", 'HI', 'IA',"ID", 'IL', 'IN',
                'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 
                'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 
                'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 
                'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 
                'TX', 'UT', 'VT', 'VA', 'WA', "VI",'WV', 'WI', 'WY']})

data_homicide=state_codes.copy()
data_homicide["Total Homicide"]=""
data_homicide.head()



,Abbreviation,Total Homicide
0,AL,
1,AK,
2,AZ,
3,AR,
4,CA,


# INFORMATION OF FBI HOMICIDE OF 2021

In [7]:
for index,row in data_homicide.iterrows():
    try:
        #with this we get the information of FBI API
        state=row["Abbreviation"]
        base_url=f"https://api.usa.gov/crime/fbi/cde/shr/state/{state}/victim/sex?from=2021&to=2021&API_KEY={api_key_FBI}"
        #print(base_url)
        response = requests.get(base_url)
        response= response.json()

        data_homicide.loc[index,"Total Homicide"]=response["data"][0]["Unknown"]+response["data"][0]["Male"]+response["data"][0]["Female"]
    except:
        data_homicide.loc[index,"Total Homicide"]="N/A"

data_homicide.head(54)
# List all 54 states in order showing the number of total Homicides   


,Abbreviation,Total Homicide
0,AL,309
1,AK,18
2,AZ,190
3,AR,321
4,CA,123
5,CO,358
6,CT,148
7,DC,109
8,DE,94
9,FL,N/A


In [24]:
# List states from least to greatest total homicide rates for each state
total_homicide_count_df = data_homicide.groupby("Total Homicide")
total_homicide_count_df.max()

,Abbreviation
Total Homicide,
6,HI
8,VT
14,NH
17,WY
18,ME
26,SD
31,MT
36,ID
38,RI


In [5]:
# Generate a bar plot showing the total number of rows 
total_homicide = total_homicide_count_df['Total Homicide'].value_counts()
total_homicide_plot = total_homicide.plot(kind='bar')

total_homicide_plot.set_xlabel('States')
total_homicide_plot.set_ylabel('Total Homicide')
total_homicide_plot.set_title('Total Homicide by State')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

NameError: name 'total_homicide_count_df' is not defined

In [25]:
# want to get the statistical overview?
total_homicide_count_df.describe()

Abbreviation                
                      count unique top freq
Total Homicide                             
6                         1      1  HI    1
8                         1      1  VT    1
14                        2      2  NH    1
17                        1      1  WY    1
18                        2      2  AK    1
26                        1      1  SD    1
31                        1      1  MT    1
36                        1      1  ID    1
38                        1      1  RI    1
70                        1      1  IA    1
85                        1      1  UT    1
87                        1      1  KS    1
94                        1      1  DE    1
95                        1      1  WV    1
109                       1      1  DC    1
123                       1      1  CA    1
124                       1      1  NY    1
132                       1      1  MA    1
138                       1      1  MD    1
139                       1      1  NJ    1
148                       1      1  CT    1
149                       1      1  MS    1
169                       1      1  NM    1
188                       1      1  OR    1
190                       1      1  AZ    1
203                       1      1  MN    1
232                       1      1  NV    1
290                       1      1  OK    1
309                       1      1  AL    1
315                       1      1  WI    1
321                       1      1  AR    1
325                       1      1  WA    1
358                       1      1  CO    1
365                       1      1  KY    1
438                       1      1  IN    1
447                       1      1  LA    1
510                       1      1  PA    1
546                       1      1  IL    1
548                       1      1  SC    1
562                       1      1  VA    1
593                       1      1  MO    1
672                       1      1  TN    1
728                       1      1  GA    1
747                       1      1  MI    1
824                       1      1  OH    1
928                       1      1  NC    1
2064                      1      1  TX    1
N/A                       5      5  FL    1

In [4]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
data_homicide = data_homicide.groupby("States")
tumor_mean = total_homicide_count_df["Total Homicide"].mean()
tumor_median = total_homicide_count_df["Total Homicide"].median()
tumor_var = total_homicide_count_df["Total Homicide"].var()
tumor_stdev = total_homicide_count_df["Total Homicide"].std()

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.
summary_df = pd.DataFrame([tumor_mean,tumor_median,tumor_var,tumor_stdev,tumor_SEM]).T
summary_df.columns = ["Homicide Mean",
                      "Homicide Median",
                      "Homicide Variance",
                      "Homicide Std. Dev."]
summary_df

NameError: name 'data_homicide' is not defined

# INFORMATION OF COST LIVING

In [76]:
cost_living_data=pd.read_csv("resources/cost-of-living-index-by-state-[updated-june-2023].csv")
cost_living_data["Rank"]=cost_living_data["2023"]
cost_living_data["State"]=cost_living_data["state"]
cost_living_data=cost_living_data.loc[:,["State","Rank"]]

cost_living_data.head()


,State,Rank
0,Hawaii,179.0
1,District of Columbia,148.7
2,Massachusetts,148.4
3,California,134.5
4,New York,125.1


# INFORMATION of HEALTH COST

In [39]:
health_data=pd.read_csv("resources/data-hGOWY.csv")
health_data.head()

,Rank,State,Score,% of adults reporting unmet need for mental health treatment in the past year who did not receive care because of cost,Health Care Spending Per Capita
0,1,South Dakota,100.00,47.10%,"$11,736"
1,2,Louisiana,86.69,32.00%,"$9,796"
2,3,West Virginia,82.31,39.50%,"$12,019"
3,4,Florida,79.51,51.90%,"$9,501"
4,5,Wyoming,78.63,64.70%,"$10,296"


In [41]:
states_USA=pd.read_csv("resources/states.csv")
states_USA.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [86]:
resume=pd.merge(states_USA,health_data[["Rank","State"]],on="State",how="inner")
resume = resume.rename(columns ={"Rank": "Healthcare Rank"})
resume.head()

,State,Abbreviation,Rank
0,Alabama,AL,28
1,Alaska,AK,12
2,Arizona,AZ,13
3,Arkansas,AR,33
4,California,CA,32


In [88]:
resume=pd.merge(resume,cost_living_data,on="State",how="inner")
resume.head()

,State,Abbreviation,Rank_x,Rank_y
0,Alabama,AL,28,88.8
1,Alaska,AK,12,124.4
2,Arizona,AZ,13,107.2
3,Arkansas,AR,33,90.3
4,California,CA,32,134.5


In [91]:
resume=pd.merge(resume,data_homicide,on="Abbreviation",how="inner")
resume.head()

,State,Abbreviation,Rank_x,Rank_y,Total Homicide
0,Alabama,AL,28,88.8,309
1,Alaska,AK,12,124.4,18
2,Arizona,AZ,13,107.2,190
3,Arkansas,AR,33,90.3,321
4,California,CA,32,134.5,123
